First, let's import the Python bindings, as usual.

In [1]:
import metapy

Now, let's download a list of stopwords and a small dataset to begin playing around with classifiers in MeTA.

In [2]:
!wget -N https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt

--2017-04-18 15:20:46--  https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2747 (2.7K) [text/plain]
Saving to: ‘lemur-stopwords.txt’

lemur-stopwords.txt 100%[===================>]   2.68K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2017-04-18 15:20:46 (90.4 MB/s) - ‘lemur-stopwords.txt’ saved [2747/2747]



In [3]:
!wget -N https://meta-toolkit.org/data/2016-01-26/ceeaus.tar.gz
!tar xvf ceeaus.tar.gz

--2017-04-18 15:20:47--  https://meta-toolkit.org/data/2016-01-26/ceeaus.tar.gz
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving meta-toolkit.org... 50.116.41.177, 2600:3c02::f03c:91ff:feae:b777
Connecting to meta-toolkit.org|50.116.41.177|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ceeaus.tar.gz’ not modified on server. Omitting download.

ceeaus/
ceeaus/ceeaus.dat.names.gz
ceeaus/ceeaus.dat.gz
ceeaus/ceeaus.dat.names
ceeaus/line.toml
ceeaus/ceeaus.dat.labels
ceeaus/ceeaus-nationality-full-corpus.txt
ceeaus/gz.toml
ceeaus/qrels.txt
ceeaus/file.toml
ceeaus/ceeaus-full-corpus.txt
ceeaus/japanese/
ceeaus/japanese/ceejus_M_smk_174.txt
ceeaus/japanese/ceejus_S_smk_090.txt
ceeaus/japanese/ceejus_L_ptj_14.txt
ceeaus/japanese/ceejus_S_ptj_024.txt
ceeaus/japanese/ceejus_M_smk_068.txt
ceeaus/japanese/ceejus_S_smk_115.txt
ceeaus/japanese/ceejus_M_ptj_128.txt
ceeaus/japanese/ceejus_M_smk_071.txt
ceeaus/japanese/ceejus_S_ptj_102.txt
ceeaus/

ceeaus/japanese/ceejus_L_smk_11.txt
ceeaus/japanese/ceejus_S_ptj_035.txt
ceeaus/japanese/ceejus_M_ptj_149.txt
ceeaus/japanese/ceejus_M_ptj_196.txt
ceeaus/japanese/ceejus_M_ptj_135.txt
ceeaus/japanese/ceejus_L_smk_06.txt
ceeaus/japanese/ceejus_M_smk_110.txt
ceeaus/japanese/ceejus_L_ptj_39.txt
ceeaus/japanese/ceejus_M_ptj_193.txt
ceeaus/japanese/ceejus_M_smk_090.txt
ceeaus/japanese/ceejus_M_ptj_047.txt
ceeaus/japanese/ceejus_M_ptj_099.txt
ceeaus/japanese/ceejus_L_smk_13.txt
ceeaus/japanese/ceejus_M_ptj_184.txt
ceeaus/japanese/ceejus_U_ptj_04.txt
ceeaus/japanese/ceejus_M_smk_176.txt
ceeaus/japanese/ceejus_S_smk_029.txt
ceeaus/japanese/ceejus_S_smk_125.txt
ceeaus/japanese/ceejus_M_smk_085.txt
ceeaus/japanese/ceejus_M_smk_104.txt
ceeaus/japanese/ceejus_S_smk_010.txt
ceeaus/japanese/ceejus_M_smk_164.txt
ceeaus/japanese/ceejus_S_smk_028.txt
ceeaus/japanese/ceejus_M_smk_149.txt
ceeaus/japanese/ceejus_M_smk_115.txt
ceeaus/japanese/ceejus_M_smk_077.txt
ceeaus/japanese/ceejus_M_ptj_086.txt
ceeaus

Now, let's index this dataset. Since we are doing classification experiments, we will most likely be concerning ourselves with a `ForwardIndex`, since we want to map document ids to their feature vector representations.

In [4]:
fidx = metapy.index.make_forward_index('ceeaus-config.toml')

Note that the feature set used for classification depends on your settings in the configuration file _at the time of indexing_. If you want to play with different feature sets, remember to change your `analyzer` pipeline in the configuration file, and also to **reindex** your documents!

Here, we've just chosen simple unigram words. This is actually a surprisingly good baseline feature set for many text classification problems.

Now that we have a `ForwardIndex` on disk, we need to load the documents we want to start playing with into memory. Since this is a small enough dataset, let's load the whole thing into memory at once.

We need to decide what kind of dataset we're using. MeTA has classes for binary classification (`BinaryDataset`) and multi-class classification (`MulticlassDataset`), which you should choose from depending on the kind of classification problem you're dealing with. Let's see how many labels we have in our corpus.

In [5]:
fidx.num_labels()

3

Since this is more than 2, we likely want a `MulticlassDataset` so we can learn a classifier that can predict which of these three labels a document should have. (But we might be interested in only determining one particular class from the rest, in which case we might actually want a `BinaryDataset`.)

For now, let's focus on the multi-class case, as that likely makes the most sense for this kind of data. Let's load or documents.

In [6]:
dset = metapy.classify.MulticlassDataset(fidx)
len(dset)

1008

We have 1008 documents, split across three labels. What are our labels?

In [7]:
set([dset.label(instance) for instance in dset])

{'chinese', 'english', 'japanese'}

This dataset is a small collection of essays written by a bunch of students with different first languages. Our goal will be to try to identify whether an essay was written by a native-Chinese speaker, a native-English speaker, or a native-Japanese speaker.

Now, because these in-memory datasets can potentially be quite large, it's beneficial to not make unnecessary copies of them to, for example, create a new list that's shuffled that contains the same documents. In most cases, you'll be operating with a `DatasetView` (either `MulticlassDatasetView` or `BinaryDatasetView`) so that you can do things like shuffle or rotate the contents of a dataset without having to actually modify it. Doing so is pretty easy: you can use Python's slicing API, or you can just construct one directly.

In [18]:
view = dset[0:len(dset)+1]
# or
view = metapy.classify.MulticlassDatasetView(dset)

Now we can, for example, shuffle this view without changing the underlying datsaet.

In [19]:
view.shuffle()
print("{} vs {}".format(view[0].id, dset[0].id))

929 vs 0


The view has been shuffled and now has documents in random order (useful in many cases to make sure that you don't have clumps of the same-labeled documents together, or to just permute the documents in a stochastic learning algorithm), but the underlying dataset is still sorted by id.

We can also use this slicing API to create a random training and testing set from our shuffled views (views also support slicing). Let's make a 75-25 split of training-testing data. (Note that's really important that we already shuffled the view!)

In [22]:
training = view[0:int(0.75*len(view))]
testing = view[int(0.75*len(view)):len(view)+1]

Now, we're ready to train a classifier! Let's start with very simple one: [Naive Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier).

In MeTA, construction of a classifier implies training of that model. Let's train a Naive Bayes classifier on our training view now.

In [26]:
nb = metapy.classify.NaiveBayes(training)

We can now classify individual documents like so.

In [27]:
nb.classify(testing[0].weights)

'english'

We might be more interested in how well we classify the testing set.

In [28]:
mtrx = nb.test(testing)
print(mtrx)


            chinese   english   japanese  
          ------------------------------
  chinese | 0.75      -         0.25      
  english | 0.0278    0.917     0.0556    
 japanese | 0.015     -         0.985     




The `test()` method of MeTA's classifiers returns to you a `ConfusionMatrix`, which contains useful information about what kinds of mistakes your classifier is making.

(Note that, due to the random shuffling, you might see different results than we do here.)

For example, we can see that this classifier seems to have some trouble with confusing native-Chinese students' essays with those of native-Japanese students. We can tell that by looking at the rows of the confusion matrix. Each row tells you what fraction of documents with that _true_ label were assigned the label for each column by the classifier. In the case of the native-Chinese label, we can see that 25% of the time they were miscategorized as being native-Japanese.

The `ConfusionMatrix` also computes a lot of metrics that are commonly used in classifier evaluation.

In [29]:
mtrx.print_stats()

------------------------------------------------------------
Class       F1 Score    Precision   Recall      Class Dist  
------------------------------------------------------------
chinese     0.75        0.75        0.75        0.0635      
english     0.957       1           0.917       0.143       
japanese    0.978       0.97        0.985       0.794       
------------------------------------------------------------
Total       0.96        0.961       0.96        
------------------------------------------------------------
252 predictions attempted, overall accuracy: 0.96



If we want to make sure that the classifier isn't overfitting to our training data, a common approach is to do [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)). Let's run CV for our Naive Bayes classifier across the whole dataset, using 5-folds, to get an idea of how well we might generalize to new data.

In [ ]:
mtrx = metapy.classify.cross_validate(lambda fold: metapy.classify.NaiveBayes(fold), view, 5)

`cross_validate()` returns a `ConfusionMatrix` just like `test()` does. We give it a function to use to create the trained classifiers for each fold, and then pass in the dataset view containing all of our documents, and the number of folds we want to use.

Let's see how we did.

In [36]:
print(mtrx)
mtrx.print_stats()


            chinese   english   japanese  
          ------------------------------
  chinese | 0.674     0.0435    0.283     
  english | 0.0137    0.945     0.0411    
 japanese | 0.0013    0.00261   0.996     


------------------------------------------------------------
Class       F1 Score    Precision   Recall      Class Dist  
------------------------------------------------------------
chinese     0.79        0.954       0.674       0.0915      
english     0.952       0.958       0.945       0.145       
japanese    0.978       0.96        0.996       0.763       
------------------------------------------------------------
Total       0.959       0.959       0.959       
------------------------------------------------------------
1005 predictions attempted, overall accuracy: 0.959



Now let's do the same thing, but for an arguably stronger baseline: [SVM](https://en.wikipedia.org/wiki/Support_vector_machine).

MeTA's implementation of SVM is actually an approximation using [stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) on the [hinge loss](https://en.wikipedia.org/wiki/Hinge_loss). It's implemented as a `BinaryClassifier`, so we will need to adapt it before it can be used to solve our multi-class clasification problem.

MeTA provides two different adapters for this scenario: [One-vs-All](https://en.wikipedia.org/wiki/Multiclass_classification#One-vs.-rest) and [One-vs-One](https://en.wikipedia.org/wiki/Multiclass_classification#One-vs.-one).

In [32]:
ova = metapy.classify.OneVsAll(training, metapy.classify.SGD, loss_id='hinge')

We construct the `OneVsAll` reduction by providing it the training documents, the name of a binary classifier, and then (as keyword arguments) any additional arguments to that chosen classifier. In this case, we use `loss_id` to specify the loss function to use.

We can now use `OneVsAll` just like any other classifier.

In [33]:
mtrx = ova.test(testing)
print(mtrx)
mtrx.print_stats()


            chinese   english   japanese  
          ------------------------------
  chinese | 0.75      -         0.25      
  english | 0.0556    0.806     0.139     
 japanese | -         0.005     0.995     


------------------------------------------------------------
Class       F1 Score    Precision   Recall      Class Dist  
------------------------------------------------------------
chinese     0.8         0.857       0.75        0.0635      
english     0.879       0.967       0.806       0.143       
japanese    0.975       0.957       0.995       0.794       
------------------------------------------------------------
Total       0.952       0.952       0.952       
------------------------------------------------------------
252 predictions attempted, overall accuracy: 0.952



In [35]:
mtrx = metapy.classify.cross_validate(lambda fold: metapy.classify.OneVsAll(fold, metapy.classify.SGD, loss_id='hinge'), view, 5)
print(mtrx)
mtrx.print_stats()


            chinese   english   japanese  
          ------------------------------
  chinese | 0.674     0.0435    0.283     
  english | 0.0137    0.945     0.0411    
 japanese | 0.0013    0.00261   0.996     


------------------------------------------------------------
Class       F1 Score    Precision   Recall      Class Dist  
------------------------------------------------------------
chinese     0.79        0.954       0.674       0.0915      
english     0.952       0.958       0.945       0.145       
japanese    0.978       0.96        0.996       0.763       
------------------------------------------------------------
Total       0.959       0.959       0.959       
------------------------------------------------------------
1005 predictions attempted, overall accuracy: 0.959



That should be enough to get you started! Try looking at `help(metapy.classify)` for a list of what's included in the bindings.